<a href="https://colab.research.google.com/github/InukshiSenarathne/chatbot_colab_testing/blob/main/model_runnner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


TEXT ANALYZER

In [ ]:
!pip install transformers

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def predict_emotion(text, model_path, class_names):
    # Load the tokenizer associated with the model
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Load the model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Make predictions
    with torch.no_grad():
        outputs = loaded_model(**inputs)
        logits = outputs.logits

    # Get the predicted class (assuming it's a classification model)
    predicted_class = torch.argmax(logits, dim=1).item()

    # Get the emotion label name
    predicted_label = class_names[predicted_class]

    # Get the confidence score (softmax probability) and round it to two decimal points
    confidence_score = round(torch.softmax(logits, dim=1)[0][predicted_class].item()*100, 2)

    return predicted_label, confidence_score

In [5]:
# Example usage:
model_path = "/content/drive/MyDrive/research_project_y4_sliit/models/finalized_models/text"  # Replace with the actual path to your saved model
class_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
text = "i'm disappointed with my career"

predicted_label, confidence_score = predict_emotion(text, model_path, class_names)
# Print the predicted label and confidence score
print(f"Predicted Emotion: {predicted_label}, Predicted accuracy: {confidence_score}%")

Predicted Emotion: sadness, Predicted accuracy: 98.06%


--------------------------------------------------------------------------

TEXT ANALYZER

In [ ]:
!pip install pandas numpy librosa matplotlib pydub tensorflow scikit-learn seaborn
!pip install moviepy pydub

In [10]:
import os
import pandas as pd
import numpy as np
import librosa
from pydub import AudioSegment, effects
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
import moviepy.editor as mp
from pydub import AudioSegment

FRAME_LENGTH = 2048
HOP_LENGTH = 512
MODEL_PATH = "/content/drive/MyDrive/research_project_y4_sliit/models/finalized_models/voice/voice.h5"

def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)

    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    padded = np.pad(trimmed, (0, 180000 - len(trimmed)), 'constant')
    return padded, sr

def test_single_audio(audio_path):
    model = load_model(MODEL_PATH)
    y, sr = preprocess_audio(audio_path)

    zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)

    # Ensure that all feature arrays have the same shape
    max_len = min(zcr.shape[1], rms.shape[1], mfccs.shape[1])
    zcr = zcr[:, :max_len]
    rms = rms[:, :max_len]
    mfccs = mfccs[:, :max_len]

    X_test = np.concatenate((zcr, rms, mfccs), axis=0)  # Concatenate along axis 0
    X_test = X_test.T  # Transpose to match the shape
    X_test = X_test.astype('float32')

    y_pred = model.predict(np.expand_dims(X_test, axis=0))
    # print(y_pred)
    emotion_labels = ['neutral', 'happy', 'sad', 'angry', 'fear', 'disgust']
    predicted_emotion = emotion_labels[np.argmax(y_pred)]
    accuracy = np.amax(y_pred)*100

    print(f"Predicted Emotion: {predicted_emotion}, Predicted accuracy: {round(accuracy, 2)}%")


In [9]:
test_single_audio("/content/drive/MyDrive/research_project_y4_sliit/voice_meeting_clips/OAF_happy/OAF_bean_happy.wav")

1/1 [==============================] - 1s 1s/step
Predicted Emotion: happy, Predicted accuracy: 99.84%


--------------------------------------------------------------------------

FACE ANALYZER